In [3]:
import numpy as np
from numpy import bincount
import sklearn
from sklearn.model_selection import StratifiedKFold, KFold

In [4]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.19.dev1.1.


In [5]:
print('Numpy Version is {}'.format(np.__version__))

Numpy Version is 1.12.0


In [6]:

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1])
skf = StratifiedKFoldMulti(n_splits=2)
skf.get_n_splits(X, y)

print(skf)
for train_index, test_index in skf.split(X, y):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]

NameError: name 'StratifiedKFoldMulti' is not defined

In [28]:
class CategoricalStratifiedKFold(StratifiedKFold):
    """Stratified Categorical output cross-validator

    Provides train/test indices to split data in train/test sets.

    This cross-validation object is a variation of StratifiedKFold that returns
    stratified folds. The folds are made by preserving the percentage of
    samples for each class. The difference is that this cross-validator work on
    categorical outputs.That is when the output is treated as a probability
    mass over the labels rather than the specific class.

    For example, the classes Man/Woman/Undefined can be represented either as
    0/1/2 or as [1 0 0]/[0 1 0]/[0 0 1], where the dimension with 1 idicates
    the class


    Parameters
    ----------
    n_splits : int, default=3
        Number of folds. Must be at least 2.

    shuffle : boolean, optional
        Whether to shuffle each stratification of the data before splitting
        into batches.

    random_state : None, int or RandomState
        When shuffle=True, pseudo-random number generator state used for
        shuffling. If None, use default numpy RNG for shuffling.

    Examples
    --------
    X = np.array([[1, 2], [3, 4], [1, 2], [3, 4],[3, 4],[3, 4]])
    y = np.array([[1,0,0], [1,0,0], [0,1,0], [0,1,0],[0,0,1],[0,0,1]])
    skf = StratifiedKFoldMulti(n_splits=2)
    skf.get_n_splits(X, y)

    print(skf)
    StratifiedKFoldMulti(n_splits=2, random_state=None, shuffle=False)

    for train_index, test_index in skf.split(X, y):
         print("TRAIN:", train_index, "TEST:", test_index)
         X_train, X_test = X[train_index], X[test_index]
         y_train, y_test = y[train_index], y[test_index]

    ('TRAIN:', array([1, 3, 5]), 'TEST:', array([0, 2, 4]))
    ('TRAIN:', array([0, 2, 4]), 'TEST:', array([1, 3, 5]))

    Notes
    -----
    All the folds have size ``trunc(n_samples / n_splits)``, the last one has
    the complementary.

    See also
    --------
    StratifiedKFold: Returns Stratified K-Folds
    RepeatedStratifiedKFold: Repeats Stratified K-Fold n times.

    """
    def _make_test_folds(self, X, y=None, groups=None):
        if self.shuffle:
            rng = check_random_state(self.random_state)
        else:
            rng = self.random_state
        y = np.asarray(y,dtype='int32')
        n_samples = y.shape[0]
        unique_y = np.array(range(y.shape[1]))
        y_counts = np.sum(y, axis=0)
        min_groups = np.min(y_counts)
        if np.all(self.n_splits > y_counts):
            raise ValueError("All the n_groups for individual classes"
                             " are less than n_splits=%d."
                             % (self.n_splits))
        if self.n_splits > min_groups:
            warnings.warn(("The least populated class in y has only %d"
                           " members, which is too few. The minimum"
                           " number of groups for any class cannot"
                           " be less than n_splits=%d."
                           % (min_groups, self.n_splits)), Warning)

        # TODO ensure y is in categorical format
        # pre-assign each sample to a test fold index using individual KFold
        # splitting strategies for each class so as to respect the balance of
        # classes
        # NOTE: Passing the data corresponding to ith class say X[y==class_i]
        # will break when the data is not 100% stratifiable for all classes.
        # So we pass np.zeroes(max(c, n_splits)) as data to the KFold
        from IPython.core.debugger import Tracer; Tracer()() 
        
        per_cls_cvs = [
            KFold(self.n_splits, shuffle=self.shuffle,
                  random_state=rng).split(np.zeros(max(count, self.n_splits)))
            for count in y_counts]

        test_folds = np.zeros(n_samples, dtype=np.int)
        for test_fold_indices, per_cls_splits in enumerate(zip(*per_cls_cvs)):
            for cls, (_, test_split) in zip(unique_y, per_cls_splits):

                # Check which of the samples have this class dimension
                test_folds_index = np.array(y == 1)[:, cls]
                cls_test_folds = test_folds[test_folds_index]
                # the test split can be too big because we used
                # KFold(...).split(X[:max(c, n_splits)]) when data is not 100%
                # stratifiable for all the classes
                # (we use a warning instead of raising an exception)
                # If this is the case, let's trim it:
                test_split = test_split[test_split < len(cls_test_folds)]
                cls_test_folds[test_split] = test_fold_indices
                test_folds[test_folds_index] = cls_test_folds

        return test_folds

In [29]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4],[3, 4],[3, 4]])
y = np.array([[1,0,0], [1,0,0], [0,1,0], [0,1,0],[0,0,1],[0,0,1]])
skf = CategoricalStratifiedKFold(n_splits=2)
skf.get_n_splits(X, y)

print(skf)
for train_index, test_index in skf.split(X, y):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]

CategoricalStratifiedKFold(n_splits=2, random_state=None, shuffle=False)
> <ipython-input-28-f84663df84b0>(89)_make_test_folds()
     87         from IPython.core.debugger import Tracer; Tracer()()
     88         per_cls_cvs = [
---> 89             KFold(self.n_splits, shuffle=self.shuffle,
     90                   random_state=rng).split(np.zeros(max(count, self.n_splits)))
     91             for count in y_counts]

ipdb> s
> <ipython-input-28-f84663df84b0>(91)_make_test_folds()
     89             KFold(self.n_splits, shuffle=self.shuffle,
     90                   random_state=rng).split(np.zeros(max(count, self.n_splits)))
---> 91             for count in y_counts]
     92 
     93         test_folds = np.zeros(n_samples, dtype=np.int)

ipdb> s
--Call--
> /home/botty/Documents/scikit-learn/sklearn/model_selection/_split.py(407)__init__()
    405     """
    406 
--> 407     def __init__(self, n_splits=3, shuffle=False,
    408                  random_state=None):
    409

In [80]:
y_train

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [1]:
%store -r ds

In [26]:
def compare(models,X,Y,scale=True):
    """Compares different sk-learn models
    
    Parameters
        ----------
    models : dict
        model object, expected ouotput dimensions
    """
    seed = 7
    np.random.seed(seed)
    results = {}
    for m in models.keys():
        
        estimators = []
        if scale:
            estimators.append(('standardize', StandardScaler()))
        estimators.append(m)
        pipeline = Pipeline(estimators)
#         kfold = CategoricalStratifiedKFold(n_splits=5, random_state=seed)
        ## set model output dimensioanlity
        Y_shaped = Y[:] if models[m] == None else Y[:,models[m]]
        if Y_shaped.shape[1] > 1:
            print "using categorical"
            kfold = CategoricalStratifiedKFold(n_splits=5, random_state=seed) 
        else:
            kfold = StratifiedKFold(n_splits=5, random_state=seed)
#         from IPython.core.debugger import Tracer; Tracer()()      
        results[m] = cross_val_score(pipeline, X, Y_shaped, cv=kfold, scoring='roc_auc',n_jobs=1)
    return results

In [ ]:
    X_sub,Y_sub = ds
    print X_sub.shape
    print Y_sub.shape
    nb_epochs = 1
    batch_size = 1e4/ X_sub.shape[1]
    m5 = ('rnn',KerasClassifier(build_fn=keras_rnn_model,input_dimensions=X_sub.shape[2],seq_len=X_sub.shape[1],hidden=100,layers=2, nb_epoch=nb_epochs, batch_size=batch_size, verbose=0))
    results = compare({m5:None},X_sub,Y_sub,scale=False)
    result

(100, 100, 44)
(100, 100, 2)
using categorical
> <ipython-input-28-f84663df84b0>(89)_make_test_folds()
     87         from IPython.core.debugger import Tracer; Tracer()()
     88         per_cls_cvs = [
---> 89             KFold(self.n_splits, shuffle=self.shuffle,
     90                   random_state=rng).split(np.zeros(max(count, self.n_splits)))
     91             for count in y_counts]

ipdb> count
*** NameError: name 'count' is not defined
ipdb> y_counts
array([[82, 18],
       [82, 18],
       [79, 21],
       [84, 16],
       [81, 19],
       [82, 18],
       [89, 11],
       [83, 17],
       [85, 15],
       [85, 15],
       [82, 18],
       [81, 19],
       [87, 13],
       [88, 12],
       [85, 15],
       [87, 13],
       [86, 14],
       [80, 20],
       [87, 13],
       [85, 15],
       [83, 17],
       [87, 13],
       [79, 21],
       [84, 16],
       [83, 17],
       [81, 19],
       [86, 14],
       [77, 23],
       [77, 23],
       [81, 19],
       [83, 17

In [19]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.models import Model
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [14]:
from keras.layers import TimeDistributed
from keras.layers.recurrent import SimpleRNN

In [15]:
def keras_rnn_model(input_dimensions,seq_len,hidden=100,layers=1):
    # create model
    model = Sequential()
    model.add(SimpleRNN(hidden, input_shape=(seq_len,input_dimensions), init='normal', activation='relu',return_sequences=True))
    if hidden is not None:
        for l in range(layers):
            model.add(SimpleRNN(hidden, init='normal', activation='relu',return_sequences=True))
    model.add(TimeDistributed(Dense(2, init='normal', activation='softmax')))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='rmsprop')
    return model